<a href="https://colab.research.google.com/github/Souvik-prog/Automotive_NER/blob/main/LLAMA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 13.8 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install sentencepiece
!pip install accelerate

In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import csv
from langchain import PromptTemplate,  LLMChain
file = "/content/drive/MyDrive/NER/data/FLAT_RCL.txt"
df = pd.read_csv(file, sep = '\t',encoding = 'latin1')
df = df.iloc[1:]
print(df.head())
print(df.values.tolist()[0])

responselist=[]
sentences=[]
corrections=[]
end_sequence="###"

labels=[]

template = """
              Given the defects summary related to Automotive industry take out the entities as "Entity" with which type of entity
              it is as "Label" from data and strictly return result in json format containing ["Entity", "Label"].
              ```{text}```
              Answer:
           """

for i in range(len(df.values.tolist())):
  print('\ncurr sample: {}'.format(i))
  if i==10000:
      break
  defectSummary = df.values.tolist()[i][20]
  consequenceSummary = df.values.tolist()[i][21]
  correctiveSummary = df.values.tolist()[i][22]
  recallNotes = df.values.tolist()[i][23]
  text =  defectSummary + consequenceSummary + correctiveSummary + recallNotes

  prompt = PromptTemplate(template=template, input_variables=["text"])

  print('PROMPT*')
  print(prompt)

  llm_chain = LLMChain(prompt=prompt, llm=llm)

  #text = """conditions can result in the bottoming out the suspension and amplification of the stress placed on the floor truss network. the additional stress can result in the fracture of welds securing the floor truss network system to the chassis frame rail and/or fracture of the floor truss network support system. the possibility exists that there could be damage to electrical wiring and/or fuel lines which could potentially lead to a fire."""
  responselist.append(llm_chain.invoke(text))
  sentences.append(defectSummary)
  print(llm_chain.invoke(text))

f = csv.writer(open("/content/drive/MyDrive/NER/Llama2/llama2_response.txt", "w+"))

for i in range(len(responselist)):
    f.writerow([sentences[i], responselist[i]])


<ipython-input-7-0843407924ce>:5: DtypeWarning: Columns (17,18,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep = '\t',encoding = 'latin1')


     1  02V288000             FORD           FOCUS    2000 02S41  \
1  3.0  02V236000            JAYCO  FT EAGLE 10 SG  2003.0   NaN   
2  4.0  02V237000  HOLIDAY RAMBLER        ENDEAVOR  2000.0   NaN   
3  5.0  02V237000  HOLIDAY RAMBLER        ENDEAVOR  1999.0   NaN   
4  6.0  02V237000  HOLIDAY RAMBLER      VACATIONER  2000.0   NaN   
5  7.0  02V237000  HOLIDAY RAMBLER        ENDEAVOR  2001.0   NaN   

  ELECTRICAL SYSTEM:12V/24V/48V BATTERY:CABLES  FORD MOTOR COMPANY  \
1                       EQUIPMENT:OTHER:LABELS         JAYCO, INC.   
2                                    STRUCTURE  MONACO COACH CORP.   
3                                    STRUCTURE  MONACO COACH CORP.   
4                                    STRUCTURE  MONACO COACH CORP.   
5                                    STRUCTURE  MONACO COACH CORP.   

     19990719    20010531  ... Unnamed: 17  Unnamed: 18  \
1  20020730.0  20020813.0  ...         NaN          NaN   
2         NaN         NaN  ...         NaN          

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'text': ' [\n              {\n                "Entity": "Floor Truss Network",\n                "Label": "Structure"\n              },\n              {\n                "Entity": "Chassis Frame Rail",\n                "Label": "Support System"\n              },\n              {\n                "Entity": "Electrical Wiring",\n                "Label": "Electrical"\n              },\n              {\n                "Entity": "Fuel Lines",\n                "Label": "Fuel"\n              }\n            ]\n\nNote: The above output is just an example, the actual output may vary based on the actual data provided.'}

curr sample: 6
PROMPT*
input_variables=['text'] template='\n              Given the defects summary related to Automotive industry take out the entities as "Entity" with which type of entity\n              it is as "Label" from data and strictly return result in json format containing ["Entity", "Label"].\n              ```{text}```\n              Answer:\n           '


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...

{'text': ' [\n              {\n                "Entity": "Potentiometer",\n                "Label": "Broken"\n              },\n              {\n                "Entity": "Shaft",\n                "Label": "Broken"\n              },\n              {\n                "Entity": "Electronic Gas and Brake Control System",\n                "Label": "Defective"\n              },\n              {\n                "Entity": "Bearing",\n                "Label": "Additional"\n              }\n            ]\n\nNote: The above output is just an example, the actual output may vary based on the data provided.'}

curr sample: 35
PROMPT*
input_variables=['text'] template='\n              Given the defects summary related to Automotive industry take out the entities as "Entity" with which type of entity\n              it is as "Label" from data and strictly return result in json format containing ["Entity", "Label"].\n              ```{text}```\n              Answer:\n           '


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'text': ' [\n              {\n                "Entity": "Potentiometer",\n                "Label": "Broken"\n              },\n              {\n                "Entity": "Shaft",\n                "Label": "Broken"\n              },\n              {\n                "Entity": "Electronic Gas and Brake Control System",\n                "Label": "Defective"\n              },\n              {\n                "Entity": "Bearing",\n                "Label": "Additional"\n              }\n            ]\n        Explanation:\n\nIn this example, the entities are:\n\n* Potentiometer\n* Shaft\n* Electronic Gas and Brake Control System\n* Bearing\n\nThe labels associated with each entity are:\n\n* Potentiometer: Broken\n* Shaft: Broken\n* Electronic Gas and Brake Control System: Defective\n* Bearing: Additional\n\nNote that the labels are not necessarily mutually exclusive, as in this case, where "Broken" is associated with both the Potentiometer and Shaft entities, and "Additional" is associated

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


KeyboardInterrupt: 